# Sample Size

In [87]:
import math
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.power import TTestIndPower
from statsmodels.stats.power import zt_ind_solve_power
from tqdm.notebook import tqdm

### Вариант 1

In [88]:
# Evan's Awesome A/B Tools - https://www.evanmiller.org/ab-testing/sample-size.html
# https://www.evanmiller.org/ab-testing/sample-size.html#!10;80;5;1;0
# Результат - 14313

### Вариант 2

In [89]:
def calc_sample_size(alpha, power, p, pct_mde, var='Absolute'):
    """ Based on https://www.evanmiller.org/ab-testing/sample-size.html

    Args:
        alpha (float): How often are you willing to accept a Type I error (false positive)?
        power (float): How often do you want to correctly detect a true positive (1-beta)?
        p (float): Base conversion rate
        pct_mde (float): Minimum detectable effect, relative to base conversion rate.

    """
    if var=='Absolute':
        delta = pct_mde
    else:
        delta = p*pct_mde

    t_alpha2 = norm.ppf(1.0-alpha/2)
    t_beta = norm.ppf(power)

    sd1 = np.sqrt(2 * p * (1.0 - p))
    sd2 = np.sqrt(p * (1.0 - p) + (p + delta) * (1.0 - p - delta))

    return (t_alpha2 * sd1 + t_beta * sd2) * (t_alpha2 * sd1 + t_beta * sd2) / (delta * delta)

In [90]:
calc_sample_size(0.05,0.8,0.1,0.01)

14312.856241916566

In [91]:
# Проверка

n = 10000
res = []
for _ in tqdm(range(n)):

  p1 = np.random.binomial(1,0.10,14313)
  p2 = np.random.binomial(1,0.11,14313)

  cnts = [p1.sum(),p2.sum()]
  nobs = [len(p1),len(p2)]
  pval = proportions_ztest(cnts,nobs)[1]
  res.append(pval <= 0.05)
np.mean(res)

  0%|          | 0/10000 [00:00<?, ?it/s]

0.7845

### Вариант 3

In [92]:
baseline_rate = 0.1
practical_significance = 0.01
confidence_level = 0.05
sensitivity = 0.8

effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + practical_significance,)
print(effect_size)

-0.03262940076737697


In [93]:
n_0, n_a = 14313, 14313   # размеры выборок
p_0, p_a = 0.1, 0.11  # доли

# дисперсии
var_0, var_a = p_0 * (1 - p_0),  p_a * (1 - p_a)

# стандартизированный размер эффекта
(p_a - p_0) / ((n_0 * np.sqrt(var_0) + n_a * np.sqrt(var_a)) / (n_0 + n_a))

0.03263229605883621

In [94]:
sms.NormalIndPower().solve_power(effect_size = effect_size,
                                 power = sensitivity,
                                 alpha = confidence_level,
                                 ratio=1)

14744.104836925611

In [95]:
TTestIndPower().solve_power(effect_size,
                            power=sensitivity,
                            nobs1=None,
                            alpha=confidence_level,
                            ratio=1
                            )

14745.065479705505

In [96]:
zt_ind_solve_power(effect_size=effect_size,
                   nobs1=None,
                   alpha=confidence_level,
                   power=sensitivity,
                   ratio=1,
                   alternative='two-sided')


14744.104836925611

### Вариант 4

Публикация "Стратификация. Как разбиение выборки повышает чувствительность A/B теста", X5RetailGroup, Хабр. Авторы Николай Назаров, Александр Сахнов

Вероятности ошибок I и II рода положим равными 0.05 и 0.20 соответственно. Допустим, для пользователей, попадающих под условия эксперимента, на исторических данных мы получили оценку средней выручки (2500 рублей) и оценку её стандартного отклонения (800 рублей). После рассылки писем будем ожидать, что выручка увеличится минимум на 100 рублей.

In [97]:
alpha = 0.05                    # вероятность ошибки I рода
beta = 0.2                      # вероятность ошибки II рода
mu_control = 2500               # средняя выручка с пользователя в контрольной группе
delta = 0.04                    # размер эффекта (100 руб = 4 процента от 2500 руб )
std = 800                       # стандартное отклонение

mu_pilot = mu_control* (1 + delta) # средняя выручка с пользователя в экспериментальной группе
effect = mu_pilot - mu_control
t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
var = 2 * std ** 2
sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
print(f'sample_size = {sample_size}')

sample_size = 1004
